# 2.5 用插件扩展答疑机器人的能力边界

## 🚄 前言
在前面的课程中，你已经能够通过优化提示词、文本解析与切片、检索过程等方法提高了答疑机器人的回复效果。但答疑机器人有一定的局限性，它似乎只可以“答疑”，而无法帮助你完成更多需求，比如帮你请假、帮你发送邮件等。即使对提示词进行精心设计、对RAG的整体流程进行优化，它依然无法做到，因为这需要与外界进行交互，而答疑机器人只能回答问题。

创建智能体（Agent）应用可以解决这些问题，它以大模型为基础，同时可以配备工具，从而达到与外界交互的目的，就像给大脑配备了四肢。

## 🍁 课程目标
学完本课程后，你将能够：

* 了解Agent的原理以及运行机制；
* 掌握 Assistant API 的基础使用方法；
* 了解Multi-Agent系统的设计方法。

## 📖 课程目录
在本节课程中，我们会向你介绍Agent如何通过集成工具函数扩展大模型的能力。

- 1.&nbsp;什么是Agent    
- 2.&nbsp;如何构建Agent    
- 3.&nbsp;Multi-Agent

## 🤖 1. 什么是Agent

一些同事希望答疑机器人能具备这样一种功能：只需说出“帮我请明天的假”，机器人便能自动提交请假申请单。

为实现此功能，答疑机器人的设计可以包括以下步骤：
1. 编写工具函数，利用公司内部的API实现请假等操作；
2. 当大模型接收到用户的指令（如“我要请后天的假”）时，解析出需要调用的工具及其参数，例如请假工具所需的日期；
3. 根据大模型的解析结果，执行相应的工具函数，并获取结果，例如“2024年x月x日至2024年y月y日的请假申请已提交”。然后直接或经过处理后将结果反馈给用户。
这种增添了工具函数的系统被称为Agent。它作为大模型与外界互动的桥梁，扩展了大模型的功能。

<img src="https://img.alicdn.com/imgextra/i4/O1CN01bJe4vz1epryTreCNG_!!6000000003921-0-tps-828-752.jpg" alt="image" style="width:500px;height:400px;">

## 🛠️ 2. 如何构建Agent
阿里云百炼平台提供了Assistant API，可以帮助你快速构建Agent。
你需要先把API Key配置到环境变量中。

In [1]:
import os
from config.load_key import load_key

load_key()
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

你配置的 API Key 是：sk-1a*****


### 2.1 定义工具函数

配置好环境变量后，我们就可以通过Assistant API来构建一个Agent了。

首先我们需要定义一些工具函数，假设我们的答疑机器人有三个功能，查询员工信息，发送请假申请，查询公司规章制度。

我们需要定义三个工具函数。

#### 2.1.1 查询员工信息

你希望将用户的提问作为工具函数的输入，并生成对应的SQL语句（这一步骤也叫NL2SQL，即自然语言转SQL语句），再使用SQL语句去数据库中查询，将查询结果作为工具函数的输出。为了帮助你更多的关注到Agent的内容，我们模拟一个查询步骤，而不实际去数据库中查询。
> 假设员工表名为employee，字段包括department（部门）、name（姓名）、HR。

> 如果你希望继续提升大模型NL2SQL的性能，请前往微调教程学习。

In [2]:
# 导入依赖
from llama_index.llms.dashscope import DashScope
from llama_index.core.base.llms.types import MessageRole, ChatMessage

# 定义一个员工查询函数
def query_employee_info(query):
    '''
    输入用户提问，输出员工信息查询结果
    '''
    # 1. 首先根据用户提问，使用NL2SQL生成SQL语句
    llm = DashScope(model_name="qwen-plus")
    messages = [
        ChatMessage(role=MessageRole.SYSTEM, content='''你有一个表叫employees，记录公司的员工信息，这个表有department（部门）、name（姓名）、HR三个字段。
    你需要根据用户输入生成sql语句进行查询,你一定不能生成sql语句之外的内容，也不要把```sql```这个信息加上。'''),
        ChatMessage(role=MessageRole.USER, content=query)
    ]
    SQL_output = llm.chat(messages).message.content
    # 打印出SQL语句
    print(f'SQL语句为：{SQL_output}')
    # 2. 根据SQL语句去查询数据库（此处为模拟查询），并返回结果
    if SQL_output == "SELECT COUNT(*) FROM employees WHERE department = '教育部门'":
        return "教育部门共有66名员工。"
    if SQL_output == "SELECT HR FROM employees WHERE name = '张三'":
        return "张三的HR是李四。"
    if SQL_output == "SELECT department FROM employees WHERE name = '王五'":
        return "王五的部门是后勤部。"
    else:
        return "抱歉，我暂时无法回答您的问题。"

# 测试一下这个函数
query_employee_info("教育部门有几个人")

SQL语句为：SELECT COUNT(*) FROM employees WHERE department = '教育部门'


'教育部门共有66名员工。'

#### 2.1.2 发送请假申请

你希望这个工具函数将员工输入的请假日期作为输入参数，并返回一个申请成功的字符串。我们在此模拟一个申请步骤，而不实际去公司系统中提交请假申请。

In [3]:
def send_leave_application(date):
    '''
    输入请假时间，输出请假申请发送结果
    '''
    return f'已为你发送请假申请，请假日期是{date}。'

# 测试一下这个函数
print(send_leave_application("明后两天"))

已为你发送请假申请，请假日期是明后两天。


#### 2.1.3 查询公司规章制度

你希望这个工具函数以用户的提问作为输入，输出是根据之前教程创建的RAG答疑机器人给出的回复。

In [4]:
from chatbot.rag import load_index

def query_company_info(query):
    '''
    输入用户提问，输出公司信息查询结果
    '''
    # 使用封装好的函数加载索引
    index = load_index()
    query_engine = index.as_query_engine(
      llm=DashScope(model_name="qwen-plus")
    )
    return query_engine.query(query).response

# 测试一下效果
query_company_info("我们公司项目管理应该用什么工具")

'公司项目管理可以使用如Jira或Trello这样的项目管理软件。这些工具能够帮助团队更好地规划和跟踪项目的进度，确保任务按时完成。同时，它们还支持任务分配、时间跟踪和团队协作等功能，有助于提高工作效率。'

尽管这些工具函数的实现比较简单，但却是大模型与外界交互的关键。接下来我们将使用Assistant API创建一个Agent，并将这三个工具函数集成进来。

### 2.2 将工具函数与大模型集成进Agent中

你已经定义好了工具函数，接下来就要将它们与大模型通过Assistant API集成到Agent中。

通过Assistants.create方法，你可以创建一个新的Agent，并通过model（模型名称）、name（Agent命名）、description（Agent的描述信息）、instructions（Agent非常重要的参数，用于提示Agent所具有的工具函数能力，同时也可以规范输出格式）、tools（工具函数通过该参数传入）参数来定义Agent。

> 其中，tools参数中的function.name用于指定工具函数，但需要为字符串格式，因此可以通过一个map方法映射到工具函数上。

In [5]:
# 引入依赖
from dashscope import Assistants, Messages, Runs, Threads
import json

# 定义公司小蜜
ChatAssistant = Assistants.create(
    # 在此指定模型名称
    model="qwen-plus",
    # 在此指定Agent名称
    name='公司小蜜',
    # 在此指定Agent的描述信息
    description='一个智能助手，能够查询员工信息，帮助员工发送请假申请，或者查询公司规章制度。',
    # 用于提示大模型所具有的工具函数能力，也可以规范输出格式
    instructions='''你是公司小蜜，你的功能有以下三个：
    1. 查询员工信息。例如：查询员工张三的HR是谁；
    2. 发送请假申请。例如：当员工提出要请假时，你可以在系统里帮他完成请假申请的发送；
    3. 查询公司规章制度。例如：我们公司项目管理的工具是什么？
    请准确判断需要调用哪个工具，并礼貌回答用户的提问。
    ''',
    # 将工具函数传入
    tools=[
        {
            # 定义工具函数类型，一般设置为function即可
            'type': 'function',
            'function': {
                # 定义工具函数名称，通过map方法映射到query_employee_info函数
                'name': '查询员工信息',
                # 定义工具函数的描述信息，Agent主要根据description来判断是否需要调用该工具函数
                'description': '当需要查询员工信息时非常有用，比如查询员工张三的HR是谁，查询教育部门总人数等。',
                # 定义工具函数的参数
                'parameters': {
                    'type': 'object',
                    'properties': {
                        # 将用户的提问作为输入参数
                        'query': {
                            'type': 'str',
                            # 对输入参数的描述
                            'description': '用户的提问。'
                        },
                    },
                    # 在此声明该工具函数需要哪些函数
                    'required': ['query']},
            }
        },
        {
            'type': 'function',
            'function': {
                'name': '发送请假申请',
                'description': '当需要帮助员工发送请假申请时非常有用。',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        # 需要请假的时间
                        'date': {
                            'type': 'str',
                            'description': '员工想要请假的时间。'
                        },
                    },
                    'required': ['date']},
            }
        },
        {
            'type': 'function',
            'function': {
                'name': '查询公司规章制度',
                'description': '当需要查询公司规章制度时非常有用，比如查询公司项目管理的工具是什么，查询公司都有哪些部门等。',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'query': {
                            'type': 'str',
                            'description': '用户的提问。'
                        }
                    }
                }
            }
        }
    ]
)
print(f'{ChatAssistant.name}创建完成')
# 建立Agent Function name与工具函数的映射关系
function_mapper = {
    "查询员工信息": query_employee_info,
    "发送请假申请": send_leave_application,
    "查询公司规章制度": query_company_info
}
print('工具函数与function.name映射关系建立完成')

公司小蜜创建完成
工具函数与function.name映射关系建立完成


通过Assistant API获得Agent回复的过程需要涉及到如thread、message、run等概念，如果你对这些概念与细节感兴趣，请参考[阿里云Assistant API官方文档](https://help.aliyun.com/zh/model-studio/developer-reference/assistantapi/)。

我们封装好了一个辅助函数`get_agent_response`，你只需要传入Agent与提问内容，即可得到指定Agent的回复。

> 如果你希望给Agent配备更多的能力，可以添加工具函数，并在function_mapper与tools中建立映射关系。

In [6]:
# 输入message信息，输出为指定Agent的回复
def get_agent_response(assistant, message=''):
    # 打印出输入Agent的信息
    thread = Threads.create()
    message = Messages.create(thread.id, content=message)
    run = Runs.create(thread.id, assistant_id=assistant.id)
    run_status = Runs.wait(run.id, thread_id=thread.id)
    # 如果响应失败，会打印出run failed
    if run_status.status == 'failed':
        print('run failed:')
    # 如果需要工具来辅助大模型输出，则进行以下流程
    if run_status.required_action:
        f = run_status.required_action.submit_tool_outputs.tool_calls[0].function
        # 获得function name
        func_name = f['name']
        # 获得function 的入参
        param = json.loads(f['arguments'])
        # 打印出工具信息
        print("function is",f)
        # 根据function name，通过function_mapper映射到函数，并将参数输入工具函数得到output输出
        if func_name in function_mapper:
            output = function_mapper[func_name](**param)
        else:
            output = ""
        tool_outputs = [{
            'output':
                output
        }]
        run = Runs.submit_tool_outputs(run.id,
                                       thread_id=thread.id,
                                       tool_outputs=tool_outputs)
        run_status = Runs.wait(run.id, thread_id=thread.id)
    run_status = Runs.get(run.id, thread_id=thread.id)
    msgs = Messages.list(thread.id)
    # 将Agent的输出返回
    return msgs['data'][0]['content'][0]['text']['value']

我们可以向ChatAssistant提问，测试一下回复效果。

In [7]:
query_stk = [
    "谁是张三的HR？",
    "教育部门一共有多少员工？",
    "王五在哪个部门？",
    "帮我提交下周三请假的申请",
    "我们公司应该用什么项目管理工具呢？"
]
for query in query_stk:
    print("提问是：")
    print(query)
    print("思考过程与最终输出是：")
    print(get_agent_response(ChatAssistant,query))
    print("\n")


提问是：
谁是张三的HR？
思考过程与最终输出是：
function is {'name': '查询员工信息', 'arguments': '{"query": "谁是张三的HR？"}', 'output': None}
SQL语句为：SELECT HR FROM employees WHERE name = '张三'
张三的HR是李四。如果您还有其他问题，欢迎随时向我提问。


提问是：
教育部门一共有多少员工？
思考过程与最终输出是：
function is {'name': '查询员工信息', 'arguments': '{"query": "教育部门总人数"}', 'output': None}
SQL语句为：SELECT COUNT(*) FROM employees WHERE department = '教育部门'
教育部门共有66名员工。如果您还有其他问题，欢迎随时向我提问。


提问是：
王五在哪个部门？
思考过程与最终输出是：
function is {'name': '查询员工信息', 'arguments': '{"query": "王五在哪个部门？"}', 'output': None}
SQL语句为：SELECT department FROM employees WHERE name = '王五'
王五在后勤部工作。如果您还有其他问题，欢迎随时向我提问。


提问是：
帮我提交下周三请假的申请
思考过程与最终输出是：
function is {'name': '发送请假申请', 'arguments': '{"date": "下周三"}', 'output': None}
已经帮您提交了下周三的请假申请，有进一步的消息我会通知您的。


提问是：
我们公司应该用什么项目管理工具呢？
思考过程与最终输出是：
function is {'name': '查询公司规章制度', 'arguments': '{"query": "我们公司项目管理的工具是什么？"}', 'output': None}
我们公司使用的项目管理工具有Jira和Trello。这些工具可以帮助团队更好地进行需求分析、任务分配和进度跟踪。如果您需要进一步的帮助或者培训，可以联系IT部门。




从测试结果可以看出，拓展了功能之后的答疑机器人达到了你预期的效果。

## 3.Multi-Agent（多智能体）

尽管答疑机器人达到了你预期的测试效果，但目前仍存在两个问题：
1. 当前工具函数较少，因此一个Agent可以完成这个任务，如果工具函数增多，一个Agent可能很难同时完成对工具的决策与对问题的回答。
2. 答疑机器人只能回答使用单一工具函数的问题，如果用户提问了一个需要使用多个工具函数的问题，那答疑机器人也将无法给出正确答案。比如：“张三的HR是谁？公司应该使用什么项目管理工具？”

In [10]:
get_agent_response(ChatAssistant,"张三的HR是谁？公司应该使用什么项目管理工具？")

function is {'name': '查询员工信息', 'arguments': '{"query": "张三的HR是谁"}', 'output': None}
SQL语句为：SELECT HR FROM employees WHERE name = '张三'


'张三的HR是谁？公司应该使用什么项目管理工具？'

由于答疑机器人目前只能完成单个工具函数的调用，因此无法返回正确结果。

为了解决这两个问题，我们可以构建一个Multi-Agent（多智能体）系统。

如果说单Agent的工作方式像一个独立开发者，那么Multi-Agent的工作方式就像一个部门，每个人都有自己的职责，相互之间按照规则进行交互。
Multi-Agent系统有多种设计思路，我们在这里介绍一个由Router Agent、若干个负责执行工具函数的Agent，以及一个Summary Agent组成的Multi-Agent系统。
- Router Agent

    根据用户的输入内容，判断要将任务分发给哪个Agent。
- 执行工具函数的Agent

    根据Router Agent分发的任务，执行属于自己的工具函数。
- Summary Agent

    根据用户的输入，以及执行工具函数的Agent的输出，生成总结并返回给用户。

### 3.1 Router Agent

Router Agent 是 Multi-Agent 系统的核心部分，它负责分析问题，并决定将任务分发到哪个Agent上。我们利用 Assistant API 创建 Router Agent，并通过 instructions 指定其可调用的 Agent 及输出格式。

由于 Agent 的返回值为字符串格式，我们要求 Router Agent 输出列表形式的字符串，例如：`["employee_info_agent", "leave_agent", "company_info_agent"]`，以便后续使用字符串解析工具将其转换为结构化数据。

In [11]:
# 决策级别的agent，决定使用哪些agent，以及它们的运行顺序
router_agent = Assistants.create(
    model="qwen-plus",
    # 定义Agent的名称
    name='流程编排机器人',
    # 定义Agent的功能描述
    description='你是团队的leader，你的手下有很多agent，你需要根据用户的输入，决定要以怎样的顺序去使用这些agent',
    # 定义对Agent的指示语句，Agent会按照指示语句进行工具的调用并返回结果。
    instructions="""你的团队中有以下agent。
    employee_info_agent：可以查询公司的员工信息，如果提问中关于部门、HR等信息，则调用该agent；
    leave_agent：可以帮助员工发送请假申请，如果用户提出请假，则调用该agent；
    company_info_agent: 可以查询公司的规章制度信息，如果用户提出关于公司规章制度的问题，则调用该agent；
    chat_agent：如果用户的问题无需以上agent，则调用该agent。

    你需要根据用户的问题，判断要以什么顺序使用这些agent，一个agent可以被多次调用。你的返回形式是一个列表，不能返回其它信息。比如：["employee_info_agent", "leave_agent","company_info_agent"]或者["chat_agent"]，列表中的元素只能为上述的agent。"""
)

print("Router Agent创建完成")

Router Agent创建完成


我们准备了几个测试问题，看下Router Agent能否分发到正确的Agent。

In [12]:
query_stk = [
    "谁是张三的HR？教育部门一共有多少员工？",
    "王五在哪个部门？帮我提交下周三请假的申请",
    "你好"
]
for query in query_stk:
    print("提问是：")
    print(query)
    print(get_agent_response(router_agent,query))
    print("\n")


提问是：
谁是张三的HR？教育部门一共有多少员工？
["employee_info_agent", "employee_info_agent"]


提问是：
王五在哪个部门？帮我提交下周三请假的申请
["employee_info_agent", "leave_agent"]


提问是：
你好
["chat_agent"]




对于这三个测试问题，Router Agent都做出了正确的选择。

Router Agent在给出了回复后，你需要将它解析为列表数据以便在后续进行处理，此处使用ast工具加载列表形式的字符串。

In [13]:
import ast

# 获得Router Agent的回复
router_response = get_agent_response(router_agent,"王五在哪个部门？帮我提交下周三请假的申请")
# 通过ast工具将列表形式的字符串加载为列表
order_stk = ast.literal_eval(router_response)
# 打印出Router Agent的规划
for i in range(len(order_stk)):
    print(f'第{i+1}步调用：{order_stk[i]}')

第1步调用：employee_info_agent
第2步调用：leave_agent


### 3.2 执行工具函数的Agent


在获取到Router Agent的规划后，我们需要进行**执行工具函数Agent**的创建。创建方法与第二节基本一致，区别在于将三个工具函数分配到三个Agent上。
> 需要确保agent变量名与Router Agent的instructions中定义的agent变量名一致。

In [14]:
# 员工信息查询agent
employee_info_agent = Assistants.create(
    model="qwen-plus",
    name='员工信息查询助手',
    description='一个智能助手，能够查询员工信息。',
    instructions='''你是员工信息查询助手，负责查询员工姓名、部门、HR等信息''',
    tools=[
        {
            'type': 'function',
            'function': {
                'name': '查询员工信息',
                'description': '当需要查询员工信息时非常有用，比如查询员工张三的HR是谁，查询教育部门总人数等。',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'query': {
                            'type': 'str',
                            'description': '用户的提问。'
                        },
                    },
                    'required': ['query']},
            }
        }
    ]
)
print(f'{employee_info_agent.name}创建完成')

# 请假申请agent
leave_agent = Assistants.create(
    model="qwen-plus",
    name='请假申请助手',
    description='一个智能助手，能够帮助员工提交请假申请。',
    instructions='''你是员工请假申请助手，负责帮助员工提交请假申请。''',
    tools=[
        {
            'type': 'function',
            'function': {
                'name': '发送请假申请',
                'description': '当需要帮助员工发送请假申请时非常有用。',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        # 需要请假的时间
                        'date': {
                            'type': 'str',
                            'description': '员工想要请假的时间。'
                        },
                    },
                    'required': ['date']},
            }
        }
    ]
)
print(f'{leave_agent.name}创建完成')

# 公司规章制度查询agent
company_info_agent =  Assistants.create(
    model="qwen-plus",
    name='公司规章制度查询助手',
    description='一个智能助手，能够帮助员工查询公司规章制度。',
    instructions='''你是公司规章制度查询助手，负责帮助员工查询公司规章制度。''',
    tools=[
        {
            'type': 'function',
            'function': {
                'name': '查询公司规章制度',
                'description': '当需要查询公司规章制度时非常有用，比如查询公司项目管理的工具是什么，查询公司都有哪些部门等。',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'query': {
                            'type': 'str',
                            'description': '用户的提问。'
                        }
                    }
                }
            }
        }
    ]
)
print(f'{company_info_agent.name}创建完成')

# 功能是回复日常问题。对于日常问题来说，可以使用价格较为低廉的模型作为agent的基座
chat_agent = Assistants.create(
    # 因为该Agent对大模型性能要求不高，因此使用成本较低的qwen-turbo模型
    model="qwen-turbo",
    name='回答日常问题的机器人',
    description='一个智能助手，解答用户的问题',
    instructions='请礼貌地回答用户的问题'
)
print(f'{chat_agent.name}创建完成')

员工信息查询助手创建完成
请假申请助手创建完成
公司规章制度查询助手创建完成
回答日常问题的机器人创建完成


### 3.3 创建Summary Agent并测试Multi-Agent效果

在完成了Router Agent和执行工具函数Agent的创建后，我们还需要创建Summary Agent，该Agent会根据用户的问题与之前Agent输出的参考信息，全面、完整地回答用户问题.

In [15]:
summary_agent = Assistants.create(
    model="qwen-plus",
    name='总结机器人',
    description='一个智能助手，根据用户的问题与参考信息，全面、完整地回答用户问题',
    instructions='你是一个智能助手，根据用户的问题与参考信息，全面、完整地回答用户问题'
)
print(f'{summary_agent.name}创建完成')

总结机器人创建完成


我们将以上步骤封装为一个get_multi_agent_response函数，这样只需要输入问题就可以得到回复了。
> 由于列表中的元素为字符串，因此通过一个agent_mapper方法将字符串格式的Agent映射到定义好的Agent对象。

In [19]:
# 将列表中的字符串映射到Agent对象上
agent_mapper = {
    "employee_info_agent": employee_info_agent,
    "leave_agent": leave_agent,
    "company_info_agent": company_info_agent
}

def get_multi_agent_response(query):
    # 获取Agent的运行顺序
    agent_order = get_agent_response(router_agent,query)
    # 由于大模型输出可能不稳定，因此加入异常处理模块处理列表字符串解析失败的问题
    try:
        order_stk = ast.literal_eval(agent_order)
        print("Router Agent正在工作：")
        for i in range(len(order_stk)):
            print(f'第{i+1}步调用：{order_stk[i]}')
        # 随着多Agent的加入，需要将Agent的输出添加到用户问题中，作为参考信息
        cur_query = query

        Agent_Message = ""
        # 依次运行Agent
        for i in range(len(order_stk)):
            cur_agent = agent_mapper[order_stk[i]]
            response = get_agent_response(cur_agent,cur_query)
            Agent_Message += f"*{order_stk[i]}*的回复为：{response}\n\n"
            # 如果当前Agent为最后一个Agent，则将其输出作为Multi Agent的输出
            if i == len(order_stk)-1:
                prompt = f"请参考已知的信息：{Agent_Message}，回答用户的问题：{query}。"
                multi_agent_response = get_agent_response(summary_agent,prompt)
                print(f'Multi-Agent回复为：{multi_agent_response}')
                return multi_agent_response
            # 如果当前Agent不是最后一个Agent，则将上一个Agent的输出response添加到下一轮的query中，作为参考信息
            else:
                # 在参考信息前后加上特殊标识符，可以防止大模型混淆参考信息与提问
                cur_query = f"你可以参考已知的信息：{response}你要完整地回答用户的问题。问题是：{query}。"
    # 兜底策略，如果上述程序运行失败，则直接调用大模型
    except Exception as e:
        return get_agent_response(chat_agent,query)

# 我们来用 “王五在哪个部门？帮我提交下周三请假的申请”进行一个测试
get_multi_agent_response("王五在哪个部门？帮我提交下周三请假的申请")

Router Agent正在工作：
第1步调用：employee_info_agent
第2步调用：leave_agent
function is {'name': '查询员工信息', 'arguments': '{"query": "王五所在部门"}', 'output': None}
SQL语句为：SELECT department FROM employees WHERE name = '王五'
function is {'name': '发送请假申请', 'arguments': '{"date": "下周三"}', 'output': None}
Multi-Agent回复为：王五在后勤部工作。关于您的请假申请，我已经帮您发送了一份申请，申请的请假日期是下周三。请您将这份请假申请提交给您的直接上级或者人力资源部门。如果有其他问题，欢迎随时向我咨询。祝您一切顺利！


'王五在后勤部工作。关于您的请假申请，我已经帮您发送了一份申请，申请的请假日期是下周三。请您将这份请假申请提交给您的直接上级或者人力资源部门。如果有其他问题，欢迎随时向我咨询。祝您一切顺利！'

## 🔥 课后小测验


【单选题】 在本教程中，Router Agent的作用是什么？（ ）

A. 负责执行工具函数

B. 负责总结Agent的输出内容

C. 负责分析用户输入，并将任务分发到其它Agent上

D. 负责打印关键日志

答案：C

## ✅ 本节小结

在本节课程中，我们探讨了如何利用插件来扩展答疑机器人的功能。为了更好地将这些知识应用于实践中，以下几点建议或许能为你提供帮助：

- **明确目标**  
  使用百炼Assistant API 创建 Agent 时，需清晰定义 Agent 的任务目标。这有助于提高设计与实施的效率。
- **系统思维**  
  构建 Multi-Agent 系统时，应考虑各 Agent 之间的交互与协作。确保系统整体运行的高效性至关重要。
- **持续迭代**  
  面对技术的快速变迁，不断学习新知并将其应用于实践，依据用户反馈对 Agent 进行调整与优化，是维持竞争力的核心。
- **全面测试**  
  在 Agent 正式部署之前，进行详尽的测试以确保其在不同环境下的稳定性和性能符合预期。